# Make DataFrame

In [3]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split

In [4]:
# Путь к файлу с аннотациями
json_file_path = 'coco_data/annotations/captions_train2017.json'

# Чтение JSON-файла
with open(json_file_path, 'r') as json_file:
    data = json.load(json_file)

# Создание списка для данных
annotations_data = {'image_path': [],
                   'caption': []}

# Обход каждой аннотации в JSON-файле
for index in range(0, 118287):
    annotations_data['image_path'].append(data['images'][index]['file_name'])
    annotations_data['caption'].append(data['annotations'][index]['caption'])


df_train = pd.DataFrame(annotations_data)

print(df_train.shape)
df_train.head()

(118287, 2)


,image_path,caption
0,000000391895.jpg,A bicycle replica with a clock as the front wh...
1,000000522418.jpg,A room with blue walls and a white sink and door.
2,000000184613.jpg,A car that seems to be parked illegally behind...
3,000000318219.jpg,A large passenger airplane flying through the ...
4,000000554625.jpg,There is a GOL plane taking off in a partly cl...


In [29]:
train_df , val_df = train_test_split(df_train, test_size = 0.1)

In [37]:
train_df.to_csv('train_data.csv')
val_df.to_csv('val_data.csv')

# Config

In [50]:
from torchvision import transforms

In [196]:
class config():
    def __init__(self):
        # model
        self.tokenizer = "cointegrated/rubert-tiny"
        self.visual = 'swin_tiny_patch4_window7_224'
        self.decoder = "cointegrated/rubert-tiny"
        
        # path
        self.train_path = "train_data.csv"
        self.train_dir = "coco_data/train2017"
        self.val_path = "val_data.csv"
        self.val_dir = "coco_data/train2017"
        
        # num
        self.learning_rate = 1e-4
        self.freeze_image_encoder = True
        self.freeze_text_encoder = False
        self.prefix_len = 77
        self.train_batch_size = 64
        self.val_batch_size = 64
        self.num_workers = 2
        self.weight_decay = 1e-4

In [197]:
config = config()
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cpu')

# Untils

In [198]:
import torch


def tokenize(tokenizer, texts, prefix_len=77):
    input_ids = []
    attention_masks = []
    for sent in texts:
        encoded_dict = tokenizer.encode_plus(
                            sent,
                            truncation = True,
                            add_special_tokens = True,
                            max_length = prefix_len,
                            # max_length = prefix_len,
                            # padding = 'max_length',
                            return_attention_mask = True,
                            return_tensors = 'pt', 
                      )
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    return input_ids, attention_masks


def _convert_image_to_rgb(image):
    return image.convert("RGB")

# Dataset

## Dataset class

In [199]:
!pip install transformers

In [100]:
!pip install open-cv

ERROR: Could not find a version that satisfies the requirement open-cv (from versions: none)
ERROR: No matching distribution found for open-cv


In [200]:
from transformers import BertTokenizer, GPT2Tokenizer
import pandas as pd
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader, random_split
import cv2
try:
    from torchvision.transforms import InterpolationMode
    BICUBIC = InterpolationMode.BICUBIC
except ImportError:
    BICUBIC = Image.BICUBIC

    
class ClipDataset(Dataset):
    
    def __init__(self, dir, df_path, transform, tokenizer, prefix_len=77):
        self.df = pd.read_csv(df_path)
        self.dir = dir
        self.prefix_len = prefix_len
        self.tokenizer = BertTokenizer.from_pretrained(tokenizer)
        self.transform = transform
        
    def __getitem__(self, idx):
        # достаем имя изображения и ее лейбл
        image_name = self.df['image_path'].iloc[idx]
        text = self.df['caption'].iloc[idx]
        input_ids, attention_mask = tokenize(self.tokenizer, [text])
        input_ids, attention_mask = input_ids[0], attention_mask[0]
        image = cv2.imread(os.path.join(self.dir, image_name))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(image)
        image = self.transform(image)
        return image, input_ids, attention_mask
    
    def __len__(self):
        return len(self.df)

## Dataset creat

In [201]:
transform = transforms.Compose([
        transforms.Resize(224, interpolation=BICUBIC),
        transforms.CenterCrop(224),
        _convert_image_to_rgb,
        transforms.ToTensor(),
        transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711)),])

transform_aug = None
        

In [202]:
train_dataset = ClipDataset(config.train_dir, config.train_path, transform, config.tokenizer, prefix_len=config.prefix_len)
val_dataset = ClipDataset(config.val_dir, config.val_path, transform, config.tokenizer, prefix_len=config.prefix_len)

In [2]:
train_dataset[0].squeeze()

NameError: name 'train_dataset' is not defined

In [204]:
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=config.train_batch_size,
                          shuffle=True,
                          pin_memory=True,
                          num_workers=config.num_workers)

In [1]:
val_loader = DataLoader(dataset=val_dataset,
                          batch_size=config.val_batch_size,
                          shuffle=False,
                          pin_memory=True,
                          num_workers=config.num_workers)

NameError: name 'DataLoader' is not defined

# Model

In [166]:
pip show urllib3


Name: urllib3
Version: 2.0.7
Summary: HTTP library with thread-safe connection pooling, file post, and more.
Home-page: 
Author: 
Author-email: Andrey Petrov <andrey.petrov@shazow.net>
License: 
Location: /Users/kekgerman/opt/anaconda3/lib/python3.9/site-packages
Requires: 
Required-by: anaconda-client, botocore, distributed, requests, sentry-sdk
Note: you may need to restart the kernel to use updated packages.


In [192]:
!pip install --upgrade transformers


In [206]:
from timm import create_model as create_swin
import numpy as np
import torch
import torch.nn.functional as F
from torch import nn
from transformers import BertModel, GPT2LMHeadModel, GPT2Tokenizer

class CaptionModel(nn.Module):
    def __init__(self,):
        super().__init__()
        self.visual = swin = create_swin(
            config.visual, pretrained=True, num_classes=0, in_chans=3) #out 768
        self.transformer = GPT2LMHeadModel.from_pretrained(config.decoder)
        self.final_ln = nn.Linear(312, 768)
        self.logit_scale = nn.Parameter(torch.ones([]) * np.log(1 / 0.07))

    @property
    def dtype(self):
        return self.visual.patch_embed.proj.weight.dtype

    def encode_image(self, image):
        return self.visual(image.type(self.dtype))

    def encode_text(self, input_ids, attention_mask):
        x = self.transformer(input_ids=input_ids, attention_mask=attention_mask)
        x = x.last_hidden_state[:, 0, :]
        x = self.final_ln(x)
        return x

    def forward(self, image, input_ids, attention_mask):
        image_features = self.encode_image(image)
        text_features = self.encode_text(input_ids, attention_mask)

        # normalized features
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)
        text_features = text_features / text_features.norm(dim=-1, keepdim=True)

        # cosine similarity as logits
        logit_scale = self.logit_scale.exp()
        logits_per_image = logit_scale * image_features @ text_features.t()
        logits_per_text = logits_per_image.t()

        return logits_per_image, logits_per_text

RuntimeError: Failed to import transformers.models.bert.modeling_bert because of the following error (look up to see its traceback):
Failed to import transformers.integrations.peft because of the following error (look up to see its traceback):
cannot import name 'DEFAULT_CIPHERS' from 'urllib3.util.ssl_' (/Users/kekgerman/opt/anaconda3/lib/python3.9/site-packages/urllib3/util/ssl_.py)

# Train

In [207]:
import torch
from transformers import get_linear_schedule_with_warmup
from tqdm.auto import tqdm

In [208]:
def train_model(model, train_loader, val_loader, loss_img, loss_txt, optimizer, scheduler, is_val, epochs_num=1, device='cuda', verbose=10):
    
    model.train()
    for epoch in range(epochs_num):
            print(f'start training epoch {epoch}')
            
            for i, batch in enumerate(tqdm(train_loader)):
                optimizer.zero_grad()
                images = batch[0].to(device)
                input_ids = batch[1].to(device)
                attention_mask = batch[2].to(device)
                
                logits_per_image, logits_per_text = model(images, input_ids, attention_mask)
                ground_truth = torch.arange(batch[1].shape[0], dtype=torch.long).to(device)
                img_l = loss_img(logits_per_image, ground_truth)
                text_l = loss_txt(logits_per_text, ground_truth)
                total_loss = (img_l + text_l)/2
                if i % verbose == 0:
                    print(f'{i}/{len(train_loader)} total_loss {total_loss}')
                total_loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 100)
                optimizer.step()
                scheduler.step()
                
            if is_val:
                print(f'start val epoch {epoch}')
                total_loss = 0
                model.eval()
                with torch.no_grad():
                    for i, batch in enumerate(tqdm(val_loader)):
                        images = batch[0].to(device)
                        input_ids = batch[1].to(device)
                        attention_mask = batch[2].to(device)
                        
                        logits_per_image, logits_per_text = model(images, input_ids, attention_mask)
                        ground_truth = torch.arange(batch[1].shape[0], dtype=torch.long).to(device)
                        img_l = loss_img(logits_per_image, ground_truth).item()
                        text_l = loss_txt(logits_per_text, ground_truth).item()
                        total_loss += (img_l + text_l)/2
                    print(f'val loss = {total_loss / len(val_loader)}')

# Main

In [209]:
model = CaptionModel().to(device)

NameError: name 'CaptionModel' is not defined

In [210]:
for i, child in enumerate(model.children()):
            if (i == 0 and self.freeze_image_encoder) or (i == 1 and self.freeze_text_encoder):
                for param in child.parameters():
                    param.requires_grad = False

NameError: name 'model' is not defined

In [211]:
loss_img = torch.nn.CrossEntropyLoss()
loss_txt = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.AdamW(model.parameters(), lr=self.learning_rate, betas=(0.9,0.98), eps=1e-8, weight_decay=self.weight_decay)
total_steps = len(train_loader) * epochs_num
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                    num_warmup_steps=0,
                                                    num_training_steps=total_steps)

NameError: name 'model' is not defined

In [ ]:
train_model(model, train_loader, val_loader, loss_img, loss_txt, optimizer, scheduler, False, epochs_num=10, device=device)